<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/PULP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [13]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path, on_bad_lines='skip')

def standardize_column_names(df):
    def standardize(col):
        col_lower = col.lower()
        if "player" in col_lower:
            return "Player"
        elif "avg" in col_lower:
            return "AVG"
        else:
            return col
    df.columns = [standardize(col) for col in df.columns]
    return df

df = standardize_column_names(df)

k = 0.02
df['Value'] = 1000 * np.exp(-k * (df['AVG'] - 1))
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)
df

,Rank,Player,Team,Bye,POS,ESPN,Sleeper,NFL,RTSports,FFC,AVG,Value
0,1.0,Christian McCaffrey,SF,9,RB,1.0,1.0,1.0,1.0,NaN,1.0,1000.000000
1,2.0,CeeDee Lamb,DAL,7,WR,3.0,3.0,2.0,2.0,NaN,2.6,968.506582
2,3.0,Tyreek Hill,MIA,6,WR,4.0,2.0,3.0,3.0,NaN,3.2,956.953957
3,4.0,Bijan Robinson,ATL,12,RB,2.0,6.0,8.0,4.0,NaN,5.0,923.116346
4,5.0,Breece Hall,NYJ,12,RB,5.0,8.0,7.0,5.0,NaN,5.4,915.760877
...,...,...,...,...,...,...,...,...,...,...,...,...
937,945.0,Mitchell Tinsley,CIN,12,WR,NaN,NaN,895.0,NaN,NaN,895.0,0.000017
938,947.0,Anthony Gould,IND,14,WR,NaN,NaN,899.0,NaN,NaN,899.0,0.000016
939,948.0,Patrick Murtagh,NaN,NaN,TE,NaN,NaN,900.0,NaN,NaN,900.0,0.000016
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df = df[['Player', 'POS', 'Value']].dropna()

# Only keep relevant positions
df = df[df['POS'].str.contains('QB|RB|WR|TE|K|DST', na=False)]

df_small = df.nlargest(220, 'Value').copy()

print(df_small['Player'].duplicated().sum())
print(df_small['POS'].value_counts())
print(df_small.head(20))

0
POS
WR     72
RB     58
QB     26
TE     25
DST    21
K      18
Name: count, dtype: int64
                 Player POS        Value
0   Christian McCaffrey  RB  1000.000000
1           CeeDee Lamb  WR   968.506582
2           Tyreek Hill  WR   956.953957
3        Bijan Robinson  RB   923.116346
4           Breece Hall  RB   915.760877
5     Amon-Ra St. Brown  WR   901.225297
6         Ja'Marr Chase  WR   894.044258
7      Justin Jefferson  WR   886.920437
8        Saquon Barkley  RB   848.742022
9            A.J. Brown  WR   831.935804
10      Jonathan Taylor  RB   828.614707
11       Garrett Wilson  WR   796.124260
12         Jahmyr Gibbs  RB   789.780674
13           Puka Nacua  WR   767.973540
14       Kyren Williams  RB   740.818221
15  Marvin Harrison Jr.  WR   729.059450
16   Travis Etienne Jr.  RB   720.363020
17        Derrick Henry  RB   711.770323
18        Isiah Pacheco  RB   681.131427
19        Davante Adams  WR   657.046820


In [4]:
!pip install pulp
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpBinary, PULP_CBC_CMD

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 17.3 MB/s eta 0:00:00


# MIP, no Draft order

In [5]:
number_of_teams = 8
teams = list(range(1, number_of_teams + 1))
min_pos = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

players = df.index.tolist()

# Modell erstellen
model = LpProblem("Fantasy_Football_Draft", LpMaximize)

# Entscheidungsvariablen x[t,i]
x = LpVariable.dicts("x", ((t, i) for t in teams for i in players), cat=LpBinary)

# Zielfunktion: Maximiere den Gesamtwert aller Teams
model += lpSum(df.loc[i, "Value"] * x[t, i] for t in teams for i in players)

# 1. Jeder Spieler kann nur von einem Team ausgewählt werden
for i in players:
    model += lpSum(x[t, i] for t in teams) <= 1

# 2. Teamgröße: maximal 15 Spieler pro Team
for t in teams:
    model += lpSum(x[t, i] for i in players) <= max_team_size

# 3. Mindestanzahl pro Position pro Team
for t in teams:
    for pos, min_count in min_pos.items():
        model += lpSum(x[t, i] for i in players if df.loc[i, "POS"] == pos) >= min_count

# Modell lösen
solver = PULP_CBC_CMD(msg=0)
model.solve(solver)

# Ergebnis ausgeben
result = {}
for t in teams:
    team_players = [i for i in players if x[t, i].varValue == 1]
    result[t] = df.loc[team_players, ["Player", "POS", "Value"]].reset_index(drop=True)

# Ausgabe: result ist ein Dictionary mit Teamnummer als Key und DataFrame als Value
for t in teams:
    print(f"\nTeam {t}:")
    print(result[t])



Team 1:
                Player  POS       Value
0     Justin Jefferson   WR  955.041962
1          CeeDee Lamb   WR  938.005000
2       Saquon Barkley   RB  932.393820
3         Nico Collins   WR  836.942423
4          Chris Olave   WR  378.325630
5       Baker Mayfield   QB  301.797203
6          Jerry Jeudy   WR  299.991841
7      Omarion Hampton   RB  252.082214
8       George Pickens   WR  239.788019
9          Evan Engram   TE  189.379943
10  Travis Etienne Jr.   RB  174.470431
11      Justin Herbert   QB  156.296568
12       Travis Hunter   WR  144.279917
13   Minnesota Vikings  DST   29.129613
14          Jake Bates    K   21.709616

Team 2:
                 Player  POS       Value
0          Trey McBride   TE  670.320046
1           Tee Higgins   WR  645.325783
2           Chase Brown   RB  510.686183
3    Kenneth Walker III   RB  434.308573
4            James Cook   RB  430.847965
5      Courtland Sutton   WR  417.279090
6    Patrick Mahomes II   QB  282.521677
7           Sa

# MIP, Snake Draft

In [6]:
# Parameter
number_of_teams = 4
max_team_size = 15
min_pos = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

teams = list(range(1, number_of_teams + 1))
players = df.index.tolist()
num_picks = max_team_size * number_of_teams

# Snake-Draft-Reihenfolge erzeugen
draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]
# draft_order[p] = Team, das bei Pick p dran ist

# MIP-Modell
model = LpProblem("Fantasy_Snake_Draft", LpMaximize)

# Entscheidungsvariablen: x[t, i, p] = 1, wenn Team t Spieler i bei Pick p wählt
x = LpVariable.dicts(
    "x",
    ((t, i, p) for t in teams for i in players for p in range(num_picks)),
    cat=LpBinary
)

# Zielfunktion: Summe der Werte aller gezogenen Spieler maximieren
model += lpSum(
    df.loc[i, "Value"] * x[t, i, p]
    for t in teams for i in players for p in range(num_picks)
)

# 1. Jeder Pick wird von genau einem Team gemacht, und das nur in der Draft-Reihenfolge
for p in range(num_picks):
    t = draft_order[p]
    model += lpSum(x[t, i, p] for i in players) == 1
    # Alle anderen Teams dürfen bei diesem Pick niemanden wählen
    for t2 in teams:
        if t2 != t:
            model += lpSum(x[t2, i, p] for i in players) == 0

# 2. Jeder Spieler wird höchstens einmal gewählt
for i in players:
    model += lpSum(x[t, i, p] for t in teams for p in range(num_picks)) <= 1

# 3. Jedes Team wählt genau max_team_size Spieler
for t in teams:
    model += lpSum(x[t, i, p] for i in players for p in range(num_picks)) == max_team_size

# 4. Mindestanzahl pro Position pro Team
for t in teams:
    for pos, min_count in min_pos.items():
        model += lpSum(
            x[t, i, p]
            for i in players if df.loc[i, "POS"] == pos
            for p in range(num_picks)
        ) >= min_count

# Modell lösen
solver = PULP_CBC_CMD(msg=1)
model.solve(solver)

# Ergebnisse sammeln: Für jeden Pick, wer wurde von wem gezogen?
draft_results = []
for p, t in enumerate(draft_order):
    for i in players:
        if x[t, i, p].varValue == 1:
            draft_results.append({
                "Pick": p + 1,
                "Team": t,
                "Player": df.loc[i, "Player"],
                "POS": df.loc[i, "POS"],
                "Value": df.loc[i, "Value"]
            })

draft_df = pd.DataFrame(draft_results)
print(draft_df)

# Optional: Team-Zusammenfassungen ausgeben
for t in teams:
    team_picks = draft_df[draft_df["Team"] == t]
    print(f"\nTeam {t}:")
    print(team_picks[["Pick", "Player", "POS", "Value"]].reset_index(drop=True))


    Pick  Team               Player  POS        Value
0      1     1   Kenneth Walker III   RB   434.308573
1      2     2        Ja'Marr Chase   WR  1000.000000
2      3     3  Christian McCaffrey   RB   707.512487
3      4     4       Jayden Daniels   QB   570.067787
4      5     4       Kyren Williams   RB   458.406011
5      6     3          Jalen Hurts   QB   473.312231
6      7     2        Derrick Henry   RB   663.650250
7      8     1      Jonathan Taylor   RB   573.498476
8      9     1          Breece Hall   RB   523.090913
9     10     2        Ladd McConkey   WR   700.472620
10    11     3       Bijan Robinson   RB   945.539136
11    12     4     Courtland Sutton   WR   417.279090
12    13     4          CeeDee Lamb   WR   938.005000
13    14     3          Rashee Rice   WR   442.196909
14    15     2        Davante Adams   WR   515.818665
15    16     1           Puka Nacua   WR   894.044258
16    17     1          Chase Brown   RB   510.686183
17    18     2             D

# MIP: Greedy Snake Draft

In [7]:
number_of_teams = 4
max_team_size = 15
min_pos = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in min_pos} for t in teams}

# Snake-Draft-Reihenfolge
draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]

available = df.copy()
draft_results = []

for pick, t in enumerate(draft_order):
    # Team-Needs bestimmen
    needs = []
    for pos, min_c in min_pos.items():
        if roster_counts[t][pos] < min_c:
            needs.append(pos)
    # Wenn alle Mindestanforderungen erfüllt, alle Positionen zulassen
    if not needs:
        needs = list(min_pos.keys())
    # Wähle den besten verfügbaren Spieler, der eine Need-Position abdeckt
    candidates = available[available["POS"].isin(needs)]
    if candidates.empty:
        candidates = available  # Falls keine Need mehr, beliebiger Spieler
    best_idx = candidates["Value"].idxmax()
    best_player = available.loc[best_idx]
    # Roster aktualisieren
    rosters[t].append(best_player["Player"])
    roster_counts[t][best_player["POS"]] += 1
    draft_results.append({
        "Pick": pick + 1,
        "Team": t,
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"]
    })
    available = available.drop(best_idx)
    # Team voll? Dann überspringen in Zukunft (optional)

draft_df = pd.DataFrame(draft_results)
print(draft_df)


    Pick  Team               Player  POS        Value
0      1     1        Ja'Marr Chase   WR  1000.000000
1      2     2     Justin Jefferson   WR   955.041962
2      3     3       Bijan Robinson   RB   945.539136
3      4     4          CeeDee Lamb   WR   938.005000
4      5     4       Saquon Barkley   RB   932.393820
5      6     3           Puka Nacua   WR   894.044258
6      7     2    Amon-Ra St. Brown   WR   876.340995
7      8     1         Jahmyr Gibbs   RB   874.590065
8      9     1         Malik Nabers   WR   860.707976
9     10     2        De'Von Achane   RB   761.854261
10    11     3         Nico Collins   WR   836.942423
11    12     4     Brian Thomas Jr.   WR   831.935804
12    13     4         Brock Bowers   TE   755.783741
13    14     3  Christian McCaffrey   RB   707.512487
14    15     2         Trey McBride   TE   670.320046
15    16     1        Derrick Henry   RB   663.650250
16    17     1           Josh Allen   QB   623.753513
17    18     2         Bucky

# MIP: B. Optional: Value Over Replacement (VOR)

In [8]:
number_of_teams = 8
roster_requirements = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

# Schritt 1: Replacement-Level pro Position bestimmen
replacement_rank = {pos: number_of_teams * req for pos, req in roster_requirements.items()}

replacement_value = {}
for pos in roster_requirements:
    pos_players = df[df['POS'] == pos].sort_values(by="Value", ascending=False).reset_index(drop=True)
    idx = replacement_rank[pos] - 1  # Nullbasiert
    if idx < len(pos_players):
        replacement_value[pos] = pos_players.loc[idx, "Value"]
    else:
        replacement_value[pos] = pos_players["Value"].min()

# Schritt 2: VOR berechnen
df["VOR"] = df.apply(lambda row: row["Value"] - replacement_value[row["POS"]], axis=1)

# Schritt 3: Snake-Draft nach VOR
teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in roster_requirements} for t in teams}

draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]

available = df.copy()
draft_results = []

for pick, t in enumerate(draft_order):
    needs = []
    for pos, min_c in roster_requirements.items():
        if roster_counts[t][pos] < min_c:
            needs.append(pos)
    if not needs:
        needs = list(roster_requirements.keys())
    candidates = available[available["POS"].isin(needs)]
    if candidates.empty:
        candidates = available
    best_idx = candidates["VOR"].idxmax()
    best_player = available.loc[best_idx]
    # Roster aktualisieren (jetzt als Dict, nicht nur Name)
    rosters[t].append({
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"]
    })
    roster_counts[t][best_player["POS"]] += 1
    draft_results.append({
        "Pick": pick + 1,
        "Team": t,
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"]
    })
    available = available.drop(best_idx)

draft_df = pd.DataFrame(draft_results)
print("Draft Board:")
print(draft_df)

# Ausgabe: Spielerlisten pro Team
print("\nSpielerlisten pro Team:")
for t in teams:
    print(f"\nTeam {t}:")
    team_df = pd.DataFrame(rosters[t])
    print(team_df.reset_index(drop=True))


Draft Board:
     Pick  Team            Player POS       Value         VOR
0       1     1      Brock Bowers  TE  755.783741  569.782141
1       2     2    Bijan Robinson  RB  945.539136  542.209058
2       3     3    Saquon Barkley  RB  932.393820  529.063742
3       4     4      Trey McBride  TE  670.320046  484.318445
4       5     5      Jahmyr Gibbs  RB  874.590065  471.259987
..    ...   ...               ...  ..         ...         ...
115   116     4       Brock Purdy  QB  147.194564  -78.629287
116   117     5    Dalton Kincaid  TE  105.188637  -80.812964
117   118     6    Caleb Williams  QB  133.453790  -92.370061
118   119     7       Jordan Love  QB  128.220992  -97.602859
119   120     8  David Montgomery  RB  304.830315  -98.499763

[120 rows x 6 columns]

Spielerlisten pro Team:

Team 1:
                Player  POS       Value         VOR
0         Brock Bowers   TE  755.783741  569.782141
1         Malik Nabers   WR  860.707976  292.915906
2         Nico Collins   WR  

# MIP: Combination Value and VOR

In [9]:
number_of_teams = 12
roster_requirements = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
max_team_size = 15

teams = list(range(1, number_of_teams + 1))
rosters = {t: [] for t in teams}
roster_counts = {t: {pos: 0 for pos in roster_requirements} for t in teams}

draft_order = []
for r in range(max_team_size):
    if r % 2 == 0:
        draft_order += teams
    else:
        draft_order += teams[::-1]

# Neue Spalte für den kombinierten Score
Value_portion = 0.8
VOR_portion = 1 - Value_portion
df["CombinedScore"] = Value_portion * df["Value"] + VOR_portion * df["VOR"]

available = df.copy()
draft_results = []

for pick, t in enumerate(draft_order):
    needs = []
    for pos, min_c in roster_requirements.items():
        if roster_counts[t][pos] < min_c:
            needs.append(pos)
    if not needs:
        needs = list(roster_requirements.keys())
    candidates = available[available["POS"].isin(needs)]
    if candidates.empty:
        candidates = available
    best_idx = candidates["CombinedScore"].idxmax()
    best_player = available.loc[best_idx]
    rosters[t].append({
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"],
        "CombinedScore": best_player["CombinedScore"]
    })
    roster_counts[t][best_player["POS"]] += 1
    draft_results.append({
        "Pick": pick + 1,
        "Team": t,
        "Player": best_player["Player"],
        "POS": best_player["POS"],
        "Value": best_player["Value"],
        "VOR": best_player["VOR"],
        "CombinedScore": best_player["CombinedScore"]
    })
    available = available.drop(best_idx)

draft_df = pd.DataFrame(draft_results)

# Spielerlisten pro Team ausgeben
for t in teams:
    print(f"\nTeam {t}:")
    team_df = pd.DataFrame(rosters[t])
    print(team_df.reset_index(drop=True))


Team 1:
            Player  POS        Value         VOR  CombinedScore
0    Ja'Marr Chase   WR  1000.000000  432.207929     886.441586
1    Ashton Jeanty   RB   621.263482  217.933404     540.597467
2      Tyreek Hill   WR   645.325783   77.533712     531.767369
3        Joe Mixon   RB   397.722800   -5.607279     317.056784
4   Baker Mayfield   QB   301.797203   75.973351     256.632433
5     Tucker Kraft   TE   120.031629  -65.969972      82.831308
6   Denver Broncos  DST    50.186962   25.709439      45.291458
7       Tyler Bass    K    11.493197   -3.293905       8.535777
8    DeVonta Smith   WR   374.561227 -193.230844     261.002813
9      C.J. Stroud   QB   116.019152 -109.804700      70.854381
10   Jakobi Meyers   WR   184.150854 -383.641217      70.592440
11   J.J. McCarthy   QB    62.411857 -163.411995      17.247087
12   Rachaad White   RB    97.881276 -305.448802      17.215261
13   Will Reichard    K     7.536480   -7.250621       4.579060
14    Joshua Karty    K     7.0

In [10]:
draft_df

,Pick,Team,Player,POS,Value,VOR,CombinedScore
0,1,1,Ja'Marr Chase,WR,1000.000000,432.207929,886.441586
1,2,2,Bijan Robinson,RB,945.539136,542.209058,864.873120
2,3,3,Saquon Barkley,RB,932.393820,529.063742,851.727804
3,4,4,Justin Jefferson,WR,955.041962,387.249891,841.483548
4,5,5,CeeDee Lamb,WR,938.005000,370.212929,824.446585
...,...,...,...,...,...,...,...
175,176,8,Tampa Bay Buccaneers,DST,7.284547,-17.192976,2.389043
176,177,9,Geno Smith,QB,47.169867,-178.653985,2.005097
177,178,10,Tank Bigsby,RB,82.413996,-320.916082,1.747981
178,179,11,Cam Little,K,4.489563,-10.297539,1.532142


# Local OPT

In [16]:
import pandas as pd
import pulp as pl

# Parameter
NUM_TEAMS = 8
MAX_TEAM_SIZE = 15
POS_REQ = {'QB':1, 'RB':2, 'WR':2, 'TE':1, 'K':1, 'DST':1}
TEAMS = list(range(1, NUM_TEAMS + 1))
PLAYERS = df_small.index.tolist()

# Snake-Draft-Reihenfolge generieren
draft_order = []
for rnd in range(MAX_TEAM_SIZE):
    draft_order += TEAMS if rnd % 2 == 0 else reversed(TEAMS)

# Modell erstellen
model = pl.LpProblem("SnakeDraft_MIP", pl.LpMaximize)

# Entscheidungsvariablen: x[p,i] = 1 wenn bei Pick p Spieler i gewählt wird
x = pl.LpVariable.dicts(
    "Pick",
    [(p, i) for p in range(len(draft_order)) for i in PLAYERS],
    cat=pl.LpBinary
)

# Zielfunktion: Gesamtwert maximieren
model += pl.lpSum(df_small.loc[i, "Value"] * x[p,i] for p,i in x)

# Constraints
# 1. Jeder Spieler maximal einmal gewählt
for i in PLAYERS:
    model += pl.lpSum(x[p,i] for p in range(len(draft_order))) <= 1

# 2. Jeder Pick wählt genau einen Spieler
for p in range(len(draft_order)):
    model += pl.lpSum(x[p,i] for i in PLAYERS) == 1

# 3. Positionsanforderungen pro Team
for t in TEAMS:
    team_picks = [p for p in range(len(draft_order)) if draft_order[p] == t]
    for pos, req in POS_REQ.items():
        pos_players = [i for i in PLAYERS if df_small.loc[i, "POS"] == pos]
        model += pl.lpSum(x[p,i] for p in team_picks for i in pos_players) >= req

# Modell lösen
solver = pl.PULP_CBC_CMD(msg=True)
model.solve(solver)

# Ergebnisse ausgeben
draft_results = []
for p in range(len(draft_order)):
    t = draft_order[p]
    for i in PLAYERS:
        if x[p,i].value() == 1:
            draft_results.append({
                'Pick': p+1,
                'Team': t,
                'Player': df_small.loc[i, 'Player'],
                'POS': df_small.loc[i, 'POS'],
                'Value': df_small.loc[i, 'Value']
            })

draft_df = pd.DataFrame(draft_results).sort_values('Pick')
print("\nDraft Verlauf:")
print(draft_df)

# Team-Roster prüfen
for t in TEAMS:
    team_df = draft_df[draft_df['Team'] == t]
    print(f"\nTeam {t} Roster:")
    print(team_df[['Player', 'POS', 'Value']].reset_index(drop=True))



Draft Verlauf:
     Pick  Team               Player  POS       Value
0       1     1          Jayden Reed   WR  178.351314
1       2     2          Zamir White   RB  269.280956
2       3     3         Dak Prescott   QB  259.759260
3       4     4  Pittsburgh Steelers  DST   42.766508
4       5     5         Tyjae Spears   RB  137.518063
..    ...   ...                  ...  ...         ...
115   116     4           DK Metcalf   WR  447.535238
116   117     5   Kenneth Walker III   RB  445.748673
117   118     6        D'Andre Swift   RB  303.613430
118   119     7       Christian Kirk   WR  229.466094
119   120     8       Brandon Aubrey    K  105.610234

[120 rows x 5 columns]

Team 1 Roster:
                Player  POS       Value
0          Jayden Reed   WR  178.351314
1       Saquon Barkley   RB  848.742022
2             DJ Moore   WR  426.560956
3            Tank Dell   WR  278.037300
4          David Njoku   TE  189.379943
5     Devin Singletary   RB  154.432218
6        DeVonta